In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder

Read data

In [2]:
df = pd.read_csv('data/sales_train_v2.csv')
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


Print dtypes and memory consumption

In [ ]:
df.info()

Downcast dtypes to save memory

In [3]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

df = downcast_dtypes(df)
#df.info()

Add aggregations

In [ ]:
'''grid = []
for date_block in df['date_block_num'].unique():
    
    # cartesian product of shops x items in each month
    cur_shops = df[df['date_block_num'] == date_block]['shop_id'].unique()
    cur_items = df[df['date_block_num'] == date_block]['item_id'].unique()
    grid.append(np.array(list(itertools.product(*[cur_shops, cur_items, [date_block]])),dtype='int32'))'''

In [ ]:
#grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [5]:
df['item_cnt_day'] = df['item_cnt_day'].clip(0,20)
gb_cnt = df.groupby(index_cols)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})
gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)

In [6]:
gb_cnt.head()

,shop_id,item_id,date_block_num,item_cnt_month
0,0,30,1,20
1,0,31,1,11
2,0,32,0,6
3,0,32,1,10
4,0,33,0,3


In [7]:
#join aggregated data to the grid

df = pd.merge(df, gb_cnt, how='left', on=index_cols).fillna(0)
df['item_cnt_month'] = df['item_cnt_month'].astype(int)
df = downcast_dtypes(df)

In [8]:
items = pd.read_csv('data/items.csv')

In [9]:
df = pd.merge(df, items[['item_id', 'item_category_id']], on = 'item_id')
#df.to_csv('train_with_grid.csv')

df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,item_category_id
0,02.01.2013,0,59,22154,999.0,1.0,1,37
1,23.01.2013,0,24,22154,999.0,1.0,1,37
2,20.01.2013,0,27,22154,999.0,1.0,1,37
3,02.01.2013,0,25,22154,999.0,1.0,5,37
4,03.01.2013,0,25,22154,999.0,1.0,5,37


In [10]:
df.to_feather('df_with_item_cnt_month')

Load data

In [14]:
items = pd.read_csv('data/items.csv')
shops = pd.read_csv('data/shops.csv')

df_test = pd.read_csv('data/test.csv', delimiter="," ,skiprows=1, usecols = (1,2), dtype=int)
df_test = pd.DataFrame(df_test, columns = ['shop_id', 'item_id'])

print('Merging with other datasets...')
# Get item category id into test_df
df_test = df_test.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')
df_test['date_block_num'] = 34
# Make testset contains same column as trainset so we can concatenate them row-wise
df_test['item_cnt_month'] = -1

df_train_test = pd.concat([df, df_test], axis = 0) 

Merging with other datasets...


/home/sfczekalski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [16]:
item_cat = pd.read_csv('data/item_categories.csv')

# Fix category
categories_list = list(item_cat.item_category_name)
for ind in range(0,1):
    categories_list[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    categories_list[ind] = 'Access'
categories_list[8] = 'Tickets (figure)'
categories_list[9] = 'Delivery of goods'
for ind in range(10,18):
    categories_list[ind] = 'Consoles'
for ind in range(18,25):
    categories_list[ind] = 'Consoles Games'
categories_list[25] = 'Accessories for games'
for ind in range(26,28):
    categories_list[ind] = 'phone games'
for ind in range(28,32):
    categories_list[ind] = 'CD games'
for ind in range(32,37):
    categories_list[ind] = 'Card'
for ind in range(37,43):
    categories_list[ind] = 'Movie'
for ind in range(43,55):
    categories_list[ind] = 'Books'
for ind in range(55,61):
    categories_list[ind] = 'Music'
for ind in range(61,73):
    categories_list[ind] = 'Gifts'
for ind in range(73,79):
    categories_list[ind] = 'Soft'
for ind in range(79,81):
    categories_list[ind] = 'Office'
for ind in range(81,83):
    categories_list[ind] = 'Clean'
categories_list[83] = 'Elements of a food'

In [18]:
df_train_test.head()

,date,date_block_num,item_cnt_day,item_cnt_month,item_id,item_price,shop_id,item_category_id
0,02.01.2013,0,1.0,1,22154.0,999.0,59.0,11.0
1,23.01.2013,0,1.0,1,22154.0,999.0,24.0,11.0
2,20.01.2013,0,1.0,1,22154.0,999.0,27.0,11.0
3,02.01.2013,0,1.0,5,22154.0,999.0,25.0,11.0
4,03.01.2013,0,1.0,5,22154.0,999.0,25.0,11.0


In [19]:
item_cat.head()

,item_category_id,item_category_name
0,14,PC Headsets / Headphones
1,0,Access
2,16,Tickets (figure)
3,8,Delivery of goods
4,6,Consoles


In [20]:
label_encoder = LabelEncoder()
item_cat['item_category_name_int'] = label_encoder.fit_transform(item_cat['item_category_name'])
item_cat.head()

,item_category_id,item_category_name,item_category_name_int
0,14,PC Headsets / Headphones,14
1,0,Access,0
2,16,Tickets (figure),16
3,8,Delivery of goods,8
4,6,Consoles,6


In [21]:
np.unique(item_cat['item_category_name_int']).size

18

In [22]:
np.unique(item_cat['item_category_name']).size

18

In [23]:
df_train_test = df_train_test.merge(item_cat, on = 'item_category_id', how = 'left')
df_train_test.head()

,date,date_block_num,item_cnt_day,item_cnt_month,item_id,item_price,shop_id,item_category_id,item_category_name,item_category_name_int
0,02.01.2013,0,1.0,1,22154.0,999.0,59.0,11.0,Movie,11.0
1,23.01.2013,0,1.0,1,22154.0,999.0,24.0,11.0,Movie,11.0
2,20.01.2013,0,1.0,1,22154.0,999.0,27.0,11.0,Movie,11.0
3,02.01.2013,0,1.0,5,22154.0,999.0,25.0,11.0,Movie,11.0
4,03.01.2013,0,1.0,5,22154.0,999.0,25.0,11.0,Movie,11.0


In [24]:
df_train_test['item_category_name'].isna().any()

True

In [25]:
df_train_test['item_category_name_int'].isna().any()

True

In [26]:
df_train_test['item_category_name'].fillna(value='Other', inplace=True)

In [27]:
df_train_test['item_category_name'].value_counts()

Movie                       836251
CD games                    506036
Consoles Games              504983
Music                       401862
Gifts                       370450
Other                       214199
Access                       76238
Books                        66589
Soft                         65234
Card                         37905
Consoles                     24770
Accessories for games        18576
Office                       10392
Elements of a food            7206
Clean                         5185
Delivery of goods             2193
Tickets (figure)              1877
phone games                     99
PC Headsets / Headphones         3
Name: item_category_name, dtype: int64

In [29]:
df_train_test['item_category_name_int'].fillna(value=1000, inplace=True)

In [30]:
df_train_test['item_category_name_int'].value_counts()

11.0      836251
3.0       506036
7.0       504983
12.0      401862
10.0      370450
1000.0    214199
0.0        76238
2.0        66589
15.0       65234
4.0        37905
6.0        24770
1.0        18576
13.0       10392
9.0         7206
5.0         5185
8.0         2193
16.0        1877
17.0          99
14.0           3
Name: item_category_name_int, dtype: int64

In [33]:
df_train_test.to_feather('df_preprocessed')

In [34]:
df_preprocessed = pd.read_feather('df_preprocessed')
df_preprocessed.head()

,date,date_block_num,item_cnt_day,item_cnt_month,item_id,item_price,shop_id,item_category_id,item_category_name,item_category_name_int
0,02.01.2013,0,1.0,1,22154.0,999.0,59.0,11.0,Movie,11.0
1,23.01.2013,0,1.0,1,22154.0,999.0,24.0,11.0,Movie,11.0
2,20.01.2013,0,1.0,1,22154.0,999.0,27.0,11.0,Movie,11.0
3,02.01.2013,0,1.0,5,22154.0,999.0,25.0,11.0,Movie,11.0
4,03.01.2013,0,1.0,5,22154.0,999.0,25.0,11.0,Movie,11.0
